In [4]:
"""
Multi-Bus Queue Simulation for University Shuttle Service
Author: Biswajit (2025)
Description:
    A discrete-event simulation that models students arriving
    at a bus stop and buses serving them at fixed intervals with limited capacity.
"""

import random
import math
from pprint import pprint

def exp_interarrival(mean):
    """Generate exponential inter-arrival time with given mean."""
    return random.expovariate(1.0 / mean)

def simulate_bus_system(
    sim_time=120,
    mean_interarrival=1.5,
    bus_interval=10,
    bus_capacity=30,
    seed=42
):
    random.seed(seed)

    time = 0.0
    queue = []
    next_student_arrival = exp_interarrival(mean_interarrival)
    next_bus_arrival = bus_interval

    total_waiting_time = 0.0
    total_served = 0
    max_queue_len = 0
    total_arrivals = 0
    total_seats_offered = 0

    while time < sim_time:
        if next_student_arrival < next_bus_arrival and next_student_arrival <= sim_time:
            time = next_student_arrival
            queue.append(time)
            total_arrivals += 1
            if len(queue) > max_queue_len:
                max_queue_len = len(queue)
            next_student_arrival = time + exp_interarrival(mean_interarrival)
        else:
            if next_bus_arrival > sim_time:
                break
            time = next_bus_arrival
            seats_left = bus_capacity
            total_seats_offered += bus_capacity
            while seats_left > 0 and len(queue) > 0:
                student_arrival_time = queue.pop(0)
                waiting = time - student_arrival_time
                total_waiting_time += waiting
                total_served += 1
                seats_left -= 1
            next_bus_arrival = time + bus_interval

    leftover_students = len(queue)
    avg_wait = total_waiting_time / total_served if total_served > 0 else 0
    bus_utilization = (total_served / total_seats_offered) if total_seats_offered > 0 else 0

    return {
        "Simulation time": sim_time,
        "Mean inter-arrival": mean_interarrival,
        "Bus interval": bus_interval,
        "Bus capacity": bus_capacity,
        "Total arrivals": total_arrivals,
        "Total served": total_served,
        "Leftover students": leftover_students,
        "Average waiting time (min)": round(avg_wait, 3),
        "Max queue length": max_queue_len,
        "Bus seat utilization": round(bus_utilization, 3)
    }

def run_scenarios():
    scenarios = [
        {"sim_time": 120, "mean_interarrival": 1.5, "bus_interval": 10, "bus_capacity": 30},
        {"sim_time": 120, "mean_interarrival": 1.5, "bus_interval": 8,  "bus_capacity": 30},
        {"sim_time": 120, "mean_interarrival": 1.5, "bus_interval": 10, "bus_capacity": 40},
    ]

    print("🚌 Multi-Bus Queue Simulation Results\n")
    results = []
    for i, sc in enumerate(scenarios, start=1):
        res = simulate_bus_system(**sc, seed=42)
        res["Scenario"] = i
        results.append(res)
        print(f"\n--- Scenario {i} ---")
        pprint(res)

    return results

if __name__ == "__main__":
    run_scenarios()

🚌 Multi-Bus Queue Simulation Results


--- Scenario 1 ---
{'Average waiting time (min)': 5.11,
 'Bus capacity': 30,
 'Bus interval': 10,
 'Bus seat utilization': 0.239,
 'Leftover students': 0,
 'Max queue length': 12,
 'Mean inter-arrival': 1.5,
 'Scenario': 1,
 'Simulation time': 120,
 'Total arrivals': 86,
 'Total served': 86}

--- Scenario 2 ---
{'Average waiting time (min)': 4.134,
 'Bus capacity': 30,
 'Bus interval': 8,
 'Bus seat utilization': 0.191,
 'Leftover students': 0,
 'Max queue length': 13,
 'Mean inter-arrival': 1.5,
 'Scenario': 2,
 'Simulation time': 120,
 'Total arrivals': 86,
 'Total served': 86}

--- Scenario 3 ---
{'Average waiting time (min)': 5.11,
 'Bus capacity': 40,
 'Bus interval': 10,
 'Bus seat utilization': 0.179,
 'Leftover students': 0,
 'Max queue length': 12,
 'Mean inter-arrival': 1.5,
 'Scenario': 3,
 'Simulation time': 120,
 'Total arrivals': 86,
 'Total served': 86}


In [1]:
# ================================
# 1) Install deps (Colab)
# ================================
# imageio + ffmpeg so we can write mp4
!pip install imageio imageio-ffmpeg --quiet


In [2]:
# ================================
# 2) Generate the animation
# ================================
import imageio.v2 as imageio
import matplotlib.pyplot as plt
import numpy as np

# video settings
duration_seconds = 4      # total length of video
fps = 24                  # frames per second
num_frames = duration_seconds * fps

# simulation-ish settings
bus_capacity = 5
initial_people = 12       # people in queue at start

frames = []

for frame in range(num_frames):
    # normalized time 0..1
    t = frame / (num_frames - 1)

    # define phases
    bus_arrive_t = 0.35   # bus moves in
    bus_leave_t = 0.7     # bus moves out

    # simple math for people + bus position
    if t < bus_arrive_t:
        # before bus arrives
        people_waiting = initial_people
        people_on_bus = 0
        # bus moves from left (-1.0) to 0.2
        bus_x = -1.0 + 1.2 * (t / bus_arrive_t)
    elif t < bus_leave_t:
        # bus is boarding
        # boarding_progress 0..1 in this interval
        boarding_progress = (t - bus_arrive_t) / (bus_leave_t - bus_arrive_t)
        # board up to capacity, gradually
        boarded = min(bus_capacity, int(boarding_progress * (bus_capacity + 1)))
        people_on_bus = boarded
        people_waiting = max(0, initial_people - boarded)
        bus_x = 0.2  # parked
    else:
        # bus leaves, final numbers
        people_on_bus = bus_capacity if initial_people >= bus_capacity else initial_people
        people_waiting = max(0, initial_people - people_on_bus)
        # move bus to the right
        bus_x = 0.2 + 1.2 * ((t - bus_leave_t) / (1 - bus_leave_t))

    # ==== draw frame ====
    fig, ax = plt.subplots(figsize=(6, 4))
    ax.set_xlim(-1.2, 2.4)
    ax.set_ylim(-0.2, 1.6)
    ax.axis('off')

    # ground line
    ax.axhline(0.2, color='black', linewidth=1)

    # draw bus
    bus_y = 0.35
    bus_w = 0.9
    bus_h = 0.6
    bus = plt.Rectangle((bus_x, bus_y), bus_w, bus_h,
                        color='#f4b41a', ec='black')
    ax.add_patch(bus)
    # door
    ax.add_patch(plt.Rectangle((bus_x + bus_w - 0.12, bus_y),
                               0.12, bus_h,
                               color='#ffeab5', ec='black', linewidth=0.5))
    # wheels
    ax.add_patch(plt.Circle((bus_x + 0.2, bus_y), 0.07, color='black'))
    ax.add_patch(plt.Circle((bus_x + 0.7, bus_y), 0.07, color='black'))
    # label
    ax.text(bus_x + bus_w/2, bus_y + bus_h/2,
            "UITS BUS", ha='center', va='center',
            fontsize=8, weight='bold')

    # draw queue (people waiting)
    start_x = -0.9
    for i in range(people_waiting):
        px = start_x - (i * 0.12)
        py = 0.28
        # body
        body = plt.Circle((px, py), 0.04, color='#467599')
        # head
        head = plt.Circle((px, py + 0.065), 0.025, color='#ffe5b4')
        ax.add_patch(body)
        ax.add_patch(head)

    # draw people on bus (heads)
    for i in range(people_on_bus):
        col = i % 5
        row = i // 5
        px = bus_x + 0.1 + col * 0.13
        py = bus_y + 0.12 + row * 0.12
        head = plt.Circle((px, py), 0.025, color='#ffe5b4')
        ax.add_patch(head)

    # info text
    ax.text(-1.1, 1.45, f"t = {t*duration_seconds:0.1f}s", fontsize=7)
    ax.text(-1.1, 1.33, f"Queue: {people_waiting}", fontsize=7)
    ax.text(-1.1, 1.21, f"On Bus: {people_on_bus}", fontsize=7)

    # convert figure to array
    fig.canvas.draw()
    w, h = fig.canvas.get_width_height()
    frame_array = np.frombuffer(fig.canvas.buffer_rgba(), dtype=np.uint8).reshape(h, w, 4)
    frames.append(frame_array)
    plt.close(fig)

# save to mp4
output_path = "multi_bus_queue_animation.mp4"
imageio.mimsave(output_path, frames, fps=fps)
print("Saved to:", output_path)


Saved to: multi_bus_queue_animation.mp4


In [3]:
# ================================
# 3) Download in Colab
# ================================
from google.colab import files
files.download("multi_bus_queue_animation.mp4")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>